#### HW1 Extract the first page of all pdfs in the "mypdfs " folder and merge them into one pdf file named "new.pdf"

In [3]:
from pathlib import Path
 
path = Path(r"mypdfs")
pdf_files = [p for p in path.glob("*.pdf")]
print(pdf_files)

from PyPDF4 import PdfFileReader, PdfFileMerger, PdfFileWriter
newfilelist = []
for file in pdf_files: 
    with open(file, "rb") as pdf_file: 
        pdf = PdfFileReader(pdf_file) 
        page = pdf.getPage(0) 
    
        writer = PdfFileWriter() 
        writer.addPage(page) 
    

        newfile = file.name[:-4]+'_new.pdf'
        newfilelist.append(newfile)
        with open(newfile, "wb") as output:
             writer.write(output)

merger = PdfFileMerger()
for file_name in newfilelist:
    merger.append(file_name)
merger.write("new.pdf")
merger.close()

import os
for file_name in newfilelist:
    os.remove(file_name)

[WindowsPath('mypdfs/1905.05406.pdf'), WindowsPath('mypdfs/1908.02426.pdf'), WindowsPath('mypdfs/2103.07579.pdf'), WindowsPath('mypdfs/2106.13112.pdf'), WindowsPath('mypdfs/2109.05673.pdf')]


#### HW2 Take the abstracts of the first 20 papers in the given URL, translate them into Chinese, and write them into the file named 'my_abstract.txt’. 
##### URL = https://openaccess.thecvf.com/ICCV2021?day=2021-10-12

In [5]:
import os
import requests
from bs4 import BeautifulSoup
import re

def get_paper_links(url):
    response = requests.get(url)
    bs = BeautifulSoup(response.text, "html.parser")
    links = bs.find_all(href=re.compile('/papers/.*.pdf'))
    paper_links = ["https://openaccess.thecvf.com" + link.get('href') for link in links[:20]]
    return paper_links

def main():
    url = "https://openaccess.thecvf.com/ICCV2021?day=2021-10-12"
    paper_links = get_paper_links(url)
    directory = "ICCV"

    if not os.path.exists(directory):
        os.makedirs(directory)

    for i, link in enumerate(paper_links):
        response = requests.get(link)
        with open(os.path.join(directory, f'paper{i+1}.pdf'), 'wb') as f:
            f.write(response.content)

if __name__ == "__main__":
    main()

In [6]:
import fitz  # PyMuPDF
from translate import Translator

def extract_pdf_abstract(pdf_path):
    # Extract text from the first page of the PDF
    doc = fitz.open(pdf_path)
    first_page = doc.load_page(0)
    text = first_page.get_text()
    doc.close()
    return text

def translate_text(text, target_language='zh-cn'):
    # Translate text using the translate library
    translator= Translator(to_lang=target_language)
    translation = translator.translate(text)
    return translation

def main():
    pdf_files = ['./ICCV/' + file for file in os.listdir('./ICCV/')]
    translated_abstracts = []


    for pdf_file in pdf_files:
        abstract = extract_pdf_abstract(pdf_file)
        translated_abstract = translate_text(abstract[:500])
        translated_abstracts.append(translated_abstract)

    # Write translated abstracts to a file
    with open('my_abstract.txt', 'w', encoding='utf-8') as file:
        for i, translated_abstract in enumerate(translated_abstracts):
            file.write(f'Paper {i + 1}:\n')
            file.write(translated_abstract + '\n\n')

if __name__ == "__main__":
    main()
